# Experiment with variables of given high correlation structure

This notebook is meant to address to a concern from a referee.

It says the following:

> Suppose
we have another predictor $x_5$, which is both correlated with $(x_1,
x_2)$ and $(x_3, x_4)$. Say $\mathrm{cor}(x_1, x_5) = 0.9$,
$\mathrm{cor}(x_2, x_5) = 0.7$, and $\mathrm{cor}(x_5, x_3)
= \mathrm{cor}(x_5, x_4) = 0.8$. Does the current method assign $x_5$
to the $(x_1, x_2)$ group or the $(x_3, x_4)$ group?

Here we investigate the problem using simulations for given correlation structure over multiple replicates.

## Correlation structure between variables

First we determine covariance matrix that satisfies correlation structure as outlined above, 

In [1]:
h = 0.92
l = 0.7
cormat = rbind(c(1.0, h,   l,   l,   0.9),
               c(h,   1.0, l,   l,   0.7),
               c(l,   l,   1.0, h,   0.8),
               c(l,   l,   h,   1.0, 0.8),
               c(0.9, 0.7, 0.8, 0.8, 1.0))

where we assume high correlation between $x_1$ and $x_2$, and $x_3$ and $x_4$ ($R_{12} = R_{34} = 0.92$). 
We set $R_{13} = R_{14} = R_{23} = R_{24} = 0.7$ and find the nearest 
positive definite matrix for this correlation structure:

In [2]:
covmat = as.matrix(Matrix::nearPD(cormat)$mat)

In [3]:
cov2cor(covmat)

1.0000000,0.9166551,0.6995185,0.6995185,0.8965043
0.9166551,1.0000000,0.6993032,0.6993032,0.7003526
0.6995185,0.6993032,1.0000000,0.9200059,0.7991611
0.6995185,0.6993032,0.9200059,1.0000000,0.7991611
0.8965043,0.7003526,0.7991611,0.7991611,1.0000000


It still satisfy the structure as outlined at the beginning of the document. We use the nearest PD found as the covariance matrix for simulation below.

## Simulation of features and response variables

We simulate an $X$ matrix of $N=600$ samples and $P_1=5$ variables, 
$$X_{p} \sim MVN(0, \Sigma)$$

where $\Sigma$ is covariance matrix as defined above.

We then expand $X$ to having a total of $P=2000$ variables where the other 1995 variables are independent --- they come from multivariate normal $MVN(0,I)$ with $I$ being the identity matrix. 

We simulate effect size $b$ a length $p$ vector with zero elsewhere except:

- Scenario 1: $x_2$ and $x_3$ are effect variables and $x_5$ is not effect variable.
- Scenario 2: $x_2$, $x_3$ and $x_5$ are all effect variables.

Non-zero effects are drawn from $N(0,1)$. Response $y$ is then generated using a linear model $y = Xb + e$, $e \sim N(0, \sigma I_n)$ with $\sigma$ chosen such that $X$ explains 20% variation in $y$.

The sample size, number of variables and effect size mimic a relatively small sample genetic association fine-mapping application.

## Analysis plan

We run 500 replicates for the above 2 scenarios. We focus our evaluation on $x_5$ and ask:

1. When $x_5$ is not a effect variable, how often is it dropped from the results, or grouped with $(x_1, x_2)$, or with $(x_3,x_4)$.
2. When $x_5$ is an effect variable, how often is it considered as a set on its own, or grouped with $(x_1, x_2)$, or with $(x_3,x_4)$.

We also assess impact on convergence issues by running SuSiE with initializations to true parameter values and see how different the result can be compared to not initialze it this way.

The code for the experiments can be found at: https://github.com/stephenslab/susie-paper/tree/master/ref_question_dsc

## Results

In [1]:
setwd('../ref_question_dsc/')
res = dscrutils::dscquery('ref4_question',targets = c('simulate.b5','analyze.cs','analyze.elbo','analyze'), module.output.files='analyze')
saveRDS(res, 'ref4_question_20200123.rds')

Calling: dsc-query ref4_question -o /tmp/jobs/65254510/RtmpsTwJwm/file886d1d05d894.csv --target "simulate.b5 analyze.cs analyze.elbo analyze" --force 
Loaded dscquery output table with 2000 rows and 6 columns.


dscquery is returning a list because one or more outputs are complex; consider converting the list to a tibble using the "tibble" package


In [2]:
res = tibble::as_tibble(readRDS('ref4_question_20200123.rds'))
head(res)

DSC,simulate.b5,analyze.cs,analyze.elbo,analyze,analyze.output.file
1,0,"1.0000000, 2.0000000, 0.9081847, 0.9540923, 0.9540923, 1.0000000, 0.9500000",-1591.2711,susie,susie/simulate_1_susie_1
1,1,"2.00, 1.00, 1.00, 1.00, 1.00, 0.95",-1543.7433,susie,susie/simulate_2_susie_1
2,0,"3.00, 1.00, 1.00, 1.00, 1.00, 0.95",-1291.7338,susie,susie/simulate_3_susie_1
2,1,0.95,-1005.7306,susie,susie/simulate_4_susie_1
3,0,"2.00, 1.00, 1.00, 1.00, 1.00, 0.95",-892.7243,susie,susie/simulate_5_susie_1
3,1,"5.00, 1.00, 1.00, 1.00, 1.00, 0.95",-1579.8866,susie,susie/simulate_6_susie_1


### When $x_5$ is not effect variable

In [15]:
out1 = res[which(res$simulate.b5==0),]

For default initialization the average ELBO is:

In [16]:
mean(out1[which(out1$analyze=='susie'),]$analyze.elbo)

[1] -1257.231

And for good initialization:

In [17]:
mean(out1[which(out1$analyze=='susie_true_init'),]$analyze.elbo)

[1] -1256.163

The ELBO improves slightly on average. For default initialization:

In [18]:
out2 = out1[which(out1$analyze=='susie'),]$analyze.cs

In [19]:
out <- sapply(out2,function (x) paste(as.character(x$cs),collapse = " and "))
out <- table(factor(out))
out <- sort(out,decreasing = TRUE)
out <- as.data.frame(out)
names(out) <- c("CSs","count")

In [20]:
out

CSs,count
2,178
3,169
2 and 3,44
3 and 2,35
2:3,19
3:4,17
1:2,11
,5
2:4,5
3 and 1:2,5


Here, variable 5 is dropped for all but one replicate, which identified one CS containing variables 1, 3, 4 and 5 (recall only $b_3$ is non-zero).

There are 5 false positive CS, and 5 replicates where no 95% CS is identified.

For "good" initialization,

In [21]:
out3 = out1[which(out1$analyze=='susie_true_init'),]$analyze.cs
out <- sapply(out3,function (x) paste(as.character(x$cs),collapse = " and "))
out <- table(factor(out))
out <- sort(out,decreasing = TRUE)
out <- as.data.frame(out)
names(out) <- c("CSs","count")

In [22]:
out

CSs,count
2 and 3,139
2,136
3,124
3 and 1:2,29
2 and 3:4,26
3:4,14
3:4 and 1:2,8
1:2,7
2 and 3:5,4
1:2 and 3:4,3


There are 4 false positive CS, and in all replicates at least one 95% CS is identified. Variable 5 are mostly dropped out, but it occasionally groups with either the first ($x_1$,$x_2$) or the second ($x_3$,$x_4$) set of variables.

### When $x_5$ is effect variable

In [29]:
out1 = res[which(res$simulate.b5!=0),]

In [30]:
mean(out1[which(out1$analyze=='susie'),]$analyze.elbo)

[1] -1433.66

In [31]:
mean(out1[which(out1$analyze=='susie_true_init'),]$analyze.elbo)

[1] -1433.416

ELBO are not very different when "good" initialization was used. For default initialization:

In [32]:
out2 = out1[which(out1$analyze=='susie'),]$analyze.cs
out <- sapply(out2,function (x) paste(as.character(x$cs),collapse = " and "))
out <- table(factor(out))
out <- sort(out,decreasing = TRUE)
out <- as.data.frame(out)
names(out) <- c("CSs","count")

In [33]:
out

CSs,count
5,88
3,86
2,82
1,46
"c(1, 5)",20
1:2,19
5 and 2,19
"c(3, 5)",15
2 and 5,14
3:4,13


For "good" initialization:

In [34]:
out2 = out1[which(out1$analyze=='susie_true_init'),]$analyze.cs
out <- sapply(out2,function (x) paste(as.character(x$cs),collapse = " and "))
out <- table(factor(out))
out <- sort(out,decreasing = TRUE)
out <- as.data.frame(out)
names(out) <- c("CSs","count")

In [35]:
out

CSs,count
2 and 3 and 5,75
5,65
2,48
2 and 5,48
3,47
2 and 3,32
3 and 5,29
2 and 5 and 3:4,14
1:2,13
1,11
